# Cycle GAN

In [1]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset
import utils
from torch.utils.data import DataLoader
import tqdm as tqdm
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler
from utils import load_csv, drop_cols, remove_strings, groupedAvg, subsample, normalize
from create_dataset import AnimalDataset
from generators import OneChannelUnetGenerator, MultiChannelUnetGenerator, SkipConnectionsMultiChannelUnetGenerator
from discriminators import OneChannelDiscriminator, MultiChannelDiscriminator 

/home/johann/anaconda3/envs/pytorch/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# config

ALLDATA = True # if False -> smaller dataset

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 512 
LEARNING_RATE = 1e-4  # 1e-5 was too small for 'LVtot_kalibriert' and 'LVtot' 
NUM_WORKERS = 10
NUM_EPOCHS = 100
LR_DECAY_AFTER_EPOCH = 250  
GENERATION_AFTER_EPOCH = NUM_EPOCHS # number of epochs after which the model generates a sample
SIG_A = "AoP"           # Drucksignal Hauptschlagader = Aortendruck
SIG_B = "VADcurrent"    # VAD Strom [A] – Pumpemstrom in Ampere
SIG_C = "VadQ"          # Fluss durch VAD (VAD = Ventrikular assistance device = Pumpe) = Pumpenfluss
SIG_D = "LVP"           # Ventrikeldruck links = Drucksignal der linken Herzkammer
TARGET = "LVtot_kalibriert"  # Kalibriertes Volumensignal der linken Herzkammer [ml] - Ventrikelvolumen
feature_names = [SIG_A]  # one signal works
CHANNELS = len(feature_names)
WINDOW = 256
EMBEDDING = False
target = TARGET

SKIPCONNECTIONS = True

# Use cycle consistency loss
CYCLE = True
LAMBDA_CYCLE = 10.0
FORWARD_CYCLE = 10.0
BACKWARD_CYCLE = 10.0
# Use supervised loss
SUPERVISED = False
LAMBDA_SUPERVISED = 10.0
# Use adversarial loss
ADVERSARIAL = False
LAMBDA_ADVERSARIAL = 100.0
# Use Identity loss
IDENTITY = True
LAMBDA_IDENTITY = 10.0

# Abailable Generators
# GENERATOR = OneChannelUnetGenerator or MultiChannelUnetGenerator or SkipConnectionsMultiChannelUnetGenerator

# Available Discminators
# DISCRIMINTOR = OneChannelDiscriminator or MultiChannelDiscriminator

LOAD_MODEL = False
SAVE_MODEL = False
SAVE_EPOCH = 100
load_from_epoch = 10  # select which Generator to load

gen_B_filename = 'Checkpoints/GenB_FeatureNames_{}_Target_[{}]_BatchSize_[{}]_LR_[{}]_Epoch_[{}].pth.tar'.format(feature_names, target, BATCH_SIZE, LEARNING_RATE, load_from_epoch)
gen_A_filename = 'Checkpoints/GenA_FeatureNames_{}_Target_[{}]_BatchSize_[{}]_LR_[{}]_Epoch_[{}].pth.tar'.format(feature_names, target, BATCH_SIZE, LEARNING_RATE, load_from_epoch)
disc_B_filename = 'Checkpoints/DiscB_FeatureNames_{}_Target_[{}]_BatchSize_[{}]_LR_[{}]_Epoch_[{}].pth.tar'.format(feature_names, target, BATCH_SIZE, LEARNING_RATE, load_from_epoch)
disc_A_filename = 'Checkpoints/DiscA_FeatureNames_{}_Target_[{}]_BatchSize_[{}]_LR_[{}]_Epoch_[{}].pth.tar'.format(feature_names, target, BATCH_SIZE, LEARNING_RATE, load_from_epoch)

CHECKPOINT_GEN_B = gen_B_filename
CHECKPOINT_GEN_A = gen_A_filename
CHECKPOINT_DISC_B =  disc_B_filename
CHECKPOINT_DISC_A =  disc_A_filename

## Dataset
### Load data from csv

In [3]:
if ALLDATA == False:
    # load only a small part of the data and drop the unnecessary columns
    # paths of only a small part of the data
    path = "/home/johann/Desktop/Uni/Masterarbeit/Cycle_GAN/LeRntVAD_csv_exports_not_all_the_data/constant_speed_interventions/" 
    df = utils.load_csv(path)
    df = utils.drop_cols(df)
    df = df.dropna()

    # select only rows where 'Phasenzuordnung' is 1
    # df = df.loc[df['Phasenzuordnung'] == 1]

    print(df.shape)  

In [4]:
if ALLDATA == True:
    # Load all the data and drop unnecessary columns
    # We load the data separately, to avoid a Runtime error

    # all the data
    path_1 = "/home/johann/Desktop/Uni/Masterarbeit/Cycle_GAN/csv_export_files_alle_Daten/csv_export_files/Data_split_1"  
    path_2 = "/home/johann/Desktop/Uni/Masterarbeit/Cycle_GAN/csv_export_files_alle_Daten/csv_export_files/Data_split_2"
    path_3 = "/home/johann/Desktop/Uni/Masterarbeit/Cycle_GAN/csv_export_files_alle_Daten/csv_export_files/Data_split_3"
    path_4 = "/home/johann/Desktop/Uni/Masterarbeit/Cycle_GAN/csv_export_files_alle_Daten/csv_export_files/Data_split_4"

    df_1 = utils.load_csv(path_1)
    df_1 = utils.drop_cols(df_1)

    df_2 = utils.load_csv(path_2)
    df_2 = utils.drop_cols(df_2)

    df_3 = utils.load_csv(path_3)
    df_3 = utils.drop_cols(df_3)

    df_4 = utils.load_csv(path_4)
    df_4 = utils.drop_cols(df_4)

    # concatenate the separate dataframes
    df = pd.concat([df_1, df_2, df_3, df_4], axis=0, ignore_index=True)
    df = df.dropna()

    print('Size of the whole dataset',df.shape)
    # select only rows where 'Phasenzuordnung' is 1
    # df = df.loc[df['Phasenzuordnung'] == 1]
    # print('Size of dataset with only the first phase',df.shape)

Size of the whole dataset (60221949, 13)


## Preprocesssing
### Removing strings from the data

In [5]:
df = utils.remove_strings(df)
# utils.visualize(df, 'AoP', 'LVP', 'VadQ', 'VADcurrent', 512)

### Subsample the data by the factor 10

In [ ]:
df = utils.subsample(df, 10)
# utils.visualize(df, 'AoP', 'LVP', 'VadQ', 'VADcurrent', 512)

### Normalize the data

In [ ]:
df = utils.normalize(df)
# utils.visualize(df, 'AoP', 'LVP', 'VadQ', 'VADcurrent', 512)

### Nicht invasive Messgroessen: 
#### Pumpenfluss, Pumpenstrom, Aortendruck , Ventrikeldruck 

***AoP*** = Drucksignal Hauptschlagader = Aortendruck

***LVP*** = Ventrikeldruck links = Drucksignal der linken Herzkammer

***VADQ*** = Fluss durch VAD (VAD = Ventrikular assistance device = Pumpe) = Pumpenfluss

***VadCurrent*** = VAD Strom [A] – Pumpemstrom in Ampere

#### Diese hier sind eventuell ebenfalls nicht invasiv

***AoQ*** = Flusssignal Hauptschlagader/ Aorta [l/min] - Aortenfluss

***VADspeed*** = VAD Geschwindigkeit [rpm] (rpm = drehzahl). Kann man nutzen

In [ ]:
#plot AoP, LVP, VadQ, VADcurrent in different plots
# utils.visualize(df, 'AoP', 'LVP', 'VadQ', 'VADcurrent', 512)

In [ ]:
# # plot AoQ and VADspeed in differents plots
# fig, axs = plt.subplots(2, 1, figsize=(20, 10))
# axs[0].plot(df['AoQ'][:5000])
# axs[0].set_title('AoQ')
# axs[1].plot(df['VADspeed'][:5000])
# axs[1].set_title('VADspeed')
# plt.show()

 ### Invasive Regelgroessen: 
#### Ventrikelvolumen, Schlagarbeit

***LVtot*** = unkalibriertes Volumensignal der linken Herzkammer [ml] (Ventrikelvolumen)

***RVtot*** = unkalibriertes Volumensignal der rechten Herzkammer [ml] (Ventrikelvolumen der rechten Herzkammer. Erstmal das der lnken anschauen)

***LVtot_kalibriert*** = kalibriertes Ventrikelvolumen

***RVtot_kalibriert*** = Kalibriertes Volumensignal der rechten Herzkammer = kalibriertes Ventrikelvolumen (wenn vorhanden – kalibriert nutzen)

In [ ]:
# plot LVtot, LVtot_kalibriert, RVtot, RVtot_kalibriert in different plots

#utils.visualize(df, 'LVtot', 'LVtot_kalibriert', 'RVtot', 'RVtot_kalibriert', 512)

### Andere relevante Variablen

***intervention*** = Gibt an welche Intervention im aktuellen Ausschnitt durchgeführt wurde: 1 = Vorlastreduktion (Googlen), 2 & 3 = Nachlasterhöhung (Googlen) , 4 = Speedramp (Pumpenstrom angehoben)

***Phasenzuordnung***

***animal***

### Split data into train and test split

In [ ]:
# hom many different intervention ids are there?
print('\nDifferent interventions: \n',df['intervention'].unique())

# hom many different animal ids are there?
print('\nDifferent animal IDs: \n',len(df['animal'].unique()))

# remove animals with less than 10 data points
df = df.groupby('animal').filter(lambda x: len(x) > 10)
print('\nDifferent animal IDs after removing those with less than 10 data points: \n',len(df['animal'].unique()))

# length of data per animal
#print(df.groupby('animal').size())

all_animals = df['animal'].unique()
# pick 2 random animals for test data
test_animals = df['animal'].sample(n=1, random_state=1).unique()

# remove test animals from train animals
train_animals =  [x for x in all_animals if x not in test_animals]

print('\nTest animals:', test_animals)
# test data
df_test = df[df['animal'].isin(test_animals)]

# train dataframe with only animals from train_animals
df_train = df[df['animal'].isin(train_animals)]
print('\nDifferent animal IDs after removing those that are in the test dataset: \n',len(df_train['animal'].unique()))


print('\nTrain data shape:', df_train.shape)
print('\nTest data shape:', df_test.shape)

#print('Unique animals in df_train: ',df_train['animal'].unique())
#print('Unique animals in df_test: ',df_test['animal'].unique())

# lengt of df_train
print('\nThe test dataset is {} percent of the whole data: '.format((len(df_test)/(len(df_train) + len(df_test))) * 100))

### Dataset loader

In [ ]:
# # Verify that the data is loaded properly

# SIG_A = "VadQ"
# SIG_B = "LVP"
# SIG_C = "AoQ"
# SIG_D = "AoP"
# feature_names = [SIG_A, SIG_B, SIG_C, SIG_D]
# target = "LVtot_kalibriert" 

# train_dataset = AnimalDataset(df_train, feature_names, target_name = target, test = False, window_length = 256)
# test_dataset = AnimalDataset(df_test, feature_names, target_name = target, test = True, window_length = 256)

# # Data loader
# train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True, num_workers=10, pin_memory=True,)
# test_loader = DataLoader(test_dataset, batch_size=256, shuffle=True, num_workers=10, pin_memory=True,)

# # length of train and test dataset
# print(len(train_dataset), len(test_dataset))

### Generator

In [ ]:
# verify OneChannelUnetGenerator generator

# x = torch.rand(4,1,256)
# model = OneChannelUnetGenerator()
# print(model(x).shape)
# gen_output = model(x)

In [ ]:
# verify MultiChannelUnetGenerator generator

# CHANNELS = 4 # Channel anpassen!!

# x1 = torch.rand(512, 1, 256)
# x2 = torch.rand(512, 1, 256)
# x3 = torch.rand(512, 1, 256)
# x4 = torch.rand(512, 1, 256)
# input = torch.cat([x1, x2, x3, x4], 1)
# # print('Input shpe: ', input.size())
# model = MultiChannelUnetGenerator(INPUTCHANNELS=CHANNELS, OUTPUTCHANNELS = 1)
# print('\nOutput for {} chanels: '.format(CHANNELS), model(input).shape)

In [ ]:
# verify SkipConnectionsMultiChannelUnetGenerator

# CHANNELS = 4 # Channel anpassen!!

# x1 = torch.rand(512, 1, 256)
# x2 = torch.rand(512, 1, 256)
# x3 = torch.rand(512, 1, 256)
# x4 = torch.rand(512, 1, 256)
# input = torch.cat([x1, x2, x3, x4], 1)
# # print('Input shpe: ', input.size())
# model = SkipConnectionsMultiChannelUnetGenerator(CHANNELS=CHANNELS)
# print('\nOutput for {} chanels: '.format(CHANNELS), model(input).shape)

### Discriminator

In [ ]:
# verify MultiChannelDiscriminator 

# CHANNELS = 4
# x1 = torch.rand(512, 1, 256)
# x2 = torch.rand(512, 1, 256)
# x3 = torch.rand(512, 1, 256)
# x4 = torch.rand(512, 1, 256)
# combined_input = torch.cat((x1, x2, x3, x4), dim=1)
# print(combined_input.shape)
# discriminator = MultiChannelDiscriminator(CHANNELS)
# print(discriminator(combined_input).shape)

In [ ]:
# verify Discriminator

# x = torch.randn((4, 1, 256))
# disc = OneChannelDiscriminator()
# print(disc(x).shape)
# print(disc(gen_output).shape)

### Train model

In [ ]:
# initialize generator and discriminator
if CHANNELS == 1:
    gen_A = OneChannelUnetGenerator().to(DEVICE)  # this uses Skipconnections
    gen_B = OneChannelUnetGenerator().to(DEVICE)
    disc_A = OneChannelDiscriminator().to(DEVICE)
    disc_B = OneChannelDiscriminator().to(DEVICE)

if CHANNELS > 1 and SKIPCONNECTIONS == False:
    gen_B = MultiChannelUnetGenerator(INPUTCHANNELS = CHANNELS, OUTPUTCHANNELS = 1).to(DEVICE)
    gen_A = MultiChannelUnetGenerator(INPUTCHANNELS = 1, OUTPUTCHANNELS = CHANNELS).to(DEVICE)
    disc_B = MultiChannelDiscriminator(CHANNELS = 1).to(DEVICE)
    disc_A = MultiChannelDiscriminator(CHANNELS = CHANNELS).to(DEVICE)

if CHANNELS > 1 and SKIPCONNECTIONS == True:
    gen_B = SkipConnectionsMultiChannelUnetGenerator(INPUTCHANNELS = CHANNELS, OUTPUTCHANNELS = 1).to(DEVICE)
    gen_A = SkipConnectionsMultiChannelUnetGenerator(INPUTCHANNELS = 1, OUTPUTCHANNELS = CHANNELS).to(DEVICE)
    disc_B = MultiChannelDiscriminator(CHANNELS = 1).to(DEVICE)
    disc_A = MultiChannelDiscriminator(CHANNELS = CHANNELS).to(DEVICE)

# optimizers for discriminator and generator 
opt_disc = torch.optim.AdamW(                                         
    list(disc_A.parameters()) + list(disc_B.parameters()), 
    lr=LEARNING_RATE, 
)
opt_gen = torch.optim.AdamW(
    list(gen_A.parameters()) + list(gen_B.parameters()),
    lr=LEARNING_RATE,
)


# maybe a step learning rate would be a good idea 1e-5 -> 1e-4
# scheduler
gen_scheduler = torch.optim.lr_scheduler.PolynomialLR(optimizer = opt_gen,
                                                      total_iters = NUM_EPOCHS-LR_DECAY_AFTER_EPOCH, 
                                                      power = 1,
                                                    )
disc_scheduler = torch.optim.lr_scheduler.PolynomialLR(optimizer = opt_disc,
                                                       total_iters = NUM_EPOCHS-LR_DECAY_AFTER_EPOCH, 
                                                       power = 1,
                                                    )
# losses
l1 = nn.L1Loss() # L1 loss for cycle consistency and identity loss
mse = nn.MSELoss() # MSE loss for adversarial loss


if LOAD_MODEL:
    utils.load_checkpoint(CHECKPOINT_GEN_B, gen_B, opt_gen, LEARNING_RATE)
    utils.load_checkpoint(CHECKPOINT_GEN_A, gen_A, opt_gen, LEARNING_RATE)
    utils.load_checkpoint(CHECKPOINT_DISC_B, disc_B, opt_disc, LEARNING_RATE)
    utils.load_checkpoint(CHECKPOINT_DISC_A, disc_A, opt_disc, LEARNING_RATE)
        


if EMBEDDING == False:
    # create datasets  
    train_dataset = AnimalDataset(df_train, feature_names, target_name = target, test = False, window_length = WINDOW)
    test_dataset = AnimalDataset(df_test, feature_names, target_name = target, test = True, window_length = WINDOW)
    gen_dataset = AnimalDataset(df_test, feature_names, target_name = target, test = False, window_length = WINDOW)

if EMBEDDING == True:
    train_dataset = AnimalDataset(df_train, feature_names, target_name = target, test = False, window_length = WINDOW)
    test_dataset = AnimalDataset(df_test, feature_names, target_name = target, test = True, window_length = WINDOW)
    gen_dataset = AnimalDataset(df_test, feature_names, target_name = target, test = False, window_length = WINDOW)

# Data loader
loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS, pin_memory=True,)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True,)
gen_loader = DataLoader(gen_dataset, batch_size=1, shuffle=True, pin_memory=True,)

# run in float16
g_scaler = torch.cuda.amp.GradScaler()
d_scaler = torch.cuda.amp.GradScaler()

train_losses = {
            'Discriminator A loss' : [],
            'Discriminator B loss' : [],
            'Total Discriminator loss' : [],
            'Adversaral loss A' : [],
            'Adversaral loss B' : [],
            'Cycle consistency loss A' : [],
            'Cycle consistency loss B' : [],
            'Total Generator loss' : [],
            'Supervised loss A' : [],
            'Supervised loss B' : [],
            'identity_loss' : [],
        }

test_losses = {
            'Discriminator A loss' : [],
            'Discriminator B loss' : [],
            'Total Discriminator loss' : [],
            'Adversaral loss A' : [],
            'Adversaral loss B' : [],
            'Cycle consistency loss A' : [],
            'Cycle consistency loss B' : [],
            'Total Generator loss' : [],
            'L1 loss between real signal A and fake signals A' : [],
            'L1 loss between real signal B and fake signals B' : [],
            'Supervised loss A' : [],
            'Supervised loss B' : [],
            'identity_loss' : [],
            'L1_realA_fakeB': [],
            'L1_realB_fakeA': [],
        }

In [ ]:
# loader.dataset.animal_dfs[0].shape[0] / 256
# (np.where(loader.dataset.animal_cumsum >= 1)[0][0])

# inputs, targets= next(iter(loader))
# print(inputs.shape, targets.shape)

# train_dataset[0][0].shape

In [ ]:
# training loop
from tqdm.auto import tqdm

for epoch in tqdm(range(NUM_EPOCHS)):

    for sig_A, sig_B in loader:
        # convert to float16
        sig_A = sig_A.float() # neccessary to prevent error: "Input type (torch.cuda.DoubleTensor) 
        sig_B = sig_B.float() # and weight type (torch.cuda.HalfTensor) should be the same"
    
        # move to GPU
        sig_A = sig_A.to(DEVICE)
        sig_B = sig_B.to(DEVICE)

        #  -------------------------------- #
        #  ----- train discriminators ----- #
        #  -------------------------------- #
        with torch.cuda.amp.autocast(): # necessary for float16
            
            # Discriminators: D_A: gen_B(A) vs. B; D_B: gen_B(A) vs. A.
            
            fake_B = gen_B(sig_A) # generate fake signal B
            d_B_real = disc_B(sig_B) # output of discriminator B for real signal B
            d_B_fake = disc_B(fake_B.detach()) # output of discriminator B for fake signal B (detached from generator)

            # B_reals += d_B_real.mean().item()
            # B_fakes += d_B_fake.mean().item()

            # Loss between dicriminator (with real signal) output and 1 - The discriminator should output 1 for real signals
            d_B_real_loss = mse(d_B_real, torch.ones_like(d_B_real))  
            # Loss between dicriminator (with fake signal) output and 0 - The discriminator should output 0 for fake signals
            d_B_fake_loss = mse(d_B_fake, torch.zeros_like(d_B_fake)) 
            # Total loss for discriminator B
            d_B_loss = d_B_real_loss + d_B_fake_loss

            fake_A = gen_A(sig_B)
            d_A_real = disc_A(sig_A)
            d_A_fake = disc_A(fake_A.detach()) 
            d_A_real_loss = mse(d_A_real, torch.ones_like(d_A_real)) 
            d_A_fake_loss = mse(d_A_fake, torch.zeros_like(d_A_fake))  
            d_A_loss = d_A_real_loss + d_A_fake_loss

            # Total loss for discriminator A
            d_loss = d_A_loss + d_B_loss  # in cycle GAN paper they halve the loss

        # exit amp.auto_cast() context manager and backpropagate 
        opt_disc.zero_grad() 
        d_scaler.scale(d_loss).backward()  
        d_scaler.step(opt_disc)  
        d_scaler.update()
        

        # -------------------------------- #
        # ------- train generators ------- #
        # -------------------------------- # 
        with torch.cuda.amp.autocast():
            
            g_loss = 0  #initialize Generator loss and add the different losses as needed

            if ADVERSARIAL:
                # ----- adversarial loss for both generators ----- #
                d_A_fake = disc_A(fake_A) # disc_A should output 0 for fake signal A
                d_B_fake = disc_B(fake_B) # disc_B should output 0 for fake signal B
                # loss between discriminator output and 0 - The discriminator should output 0 for fake signals
                g_A_loss = mse(d_A_fake, torch.ones_like(d_A_fake))   
                g_B_loss = mse(d_B_fake, torch.ones_like(d_B_fake)) 

                g_loss += LAMBDA_ADVERSARIAL * (g_A_loss + g_B_loss)

            if SUPERVISED:
                # supervised A loss
                sup_A_loss = l1(sig_A, fake_A)
                sup_B_loss = l1(sig_B, fake_B)

                g_loss += LAMBDA_SUPERVISED * (sup_B_loss  + sup_A_loss) 
                

            if CYCLE:
                # ----- cycle consistency loss ----- #
                
                # Forward cycle loss:  lambda_A * ||G_B(G_A(A)) - A|| (Eqn. (2) in the paper)
                
                cycle_A_loss = l1(gen_A(gen_B(sig_A)), sig_A)  # forward cycle loss
                cycle_B_loss = l1(gen_B(gen_A(sig_B)), sig_B)  # Backward cycle loss
                
                # cycle_B = gen_B(fake_A) # fake_A = gen_A(sig_B)  
                # cycle_A = gen_A(fake_B) # fake_B = gen_B(sig_A)
                # cycle_B_loss = l1(sig_B, cycle_B)  # l1 loss: Mean absolute error between each element in the input x and target y.
                # cycle_A_loss = l1(sig_A, cycle_A)

                g_loss += LAMBDA_CYCLE * (BACKWARD_CYCLE * cycle_B_loss + FORWARD_CYCLE * cycle_A_loss) 

            if IDENTITY:
            # ----- identity loss ----- #
            
            # Identity loss (optional): lambda_identity * (||G_A(B) - B|| * lambda_B + ||G_B(A) - A|| * lambda_A) 
            # (Sec 5.2 " Photo generation from paintings" in the paper)
                
                identity_loss = LAMBDA_IDENTITY * (LAMBDA_CYCLE * (l1(gen_B(sig_B), sig_B) + l1(gen_A(sig_A), sig_A)))
                g_loss += identity_loss
                # id_B = gen_B(sig_B) 
                # id_A = gen_A(sig_A)
                # id_B_loss = l1(sig_A, id_B)
                # id_A_loss = l1(sig_B, id_A)

                # g_loss += id_B_loss * LAMBDA_IDENTITY + id_A_loss * LAMBDA_IDENTITY

        # update gradients of generator
        opt_gen.zero_grad()
        g_scaler.scale(g_loss).backward() 
        g_scaler.step(opt_gen) 
        g_scaler.update()

    # save losses
    train_losses['Discriminator A loss'].append(d_A_loss.item())
    train_losses['Discriminator B loss'].append(d_B_loss.item())
    train_losses['Total Discriminator loss'].append(d_loss.item())
    if ADVERSARIAL:
        train_losses['Adversaral loss A'].append(g_A_loss.item())
        train_losses['Adversaral loss B'].append(g_B_loss.item())
    if CYCLE: 
        train_losses['Cycle consistency loss A'].append(cycle_A_loss.item())
        train_losses['Cycle consistency loss B'].append(cycle_B_loss.item())
    if SUPERVISED: 
        train_losses['Supervised loss A'].append(sup_A_loss.item())
        train_losses['Supervised loss B'].append(sup_B_loss.item())
    
    if IDENTITY:
        train_losses['identity_loss'].append(identity_loss.item())
        # train_losses['Identity loss A'].append(id_A_loss.item())
        # train_losses['Identity loss B'].append(id_B_loss.item())
        
    train_losses['Total Generator loss'].append(g_loss.item())
    

    # ------------------------ #
    # ------ Validation ------ #
    # ------------------------ #

    #  validation every 10 epochs
    if (epoch+1) % 10 == 0:

        with torch.no_grad():
            # set models to evaluation mode
            disc_A.eval()  # set discriminator to evaluation mode
            disc_B.eval()  # turns off Dropouts Layers, BatchNorm Layers etc
            gen_B.eval()
            gen_A.eval()

            # store losses for testing
            test_Discrminator_A_loss = 0 #
            test_Discrminator_B_loss = 0 #
            test_Total_Discrminator_loss = 0 #
            test_Adversaral_loss_A = 0  #
            test_Adversaral_loss_B = 0  #
            test_Cycle_consistency_loss_A = 0 #
            test_Cycle_consistency_loss_B = 0 #
            test_sup_A_loss = 0
            test_sup_B_loss = 0
            test_Total_Generator_loss = 0 #
            test_L1_real_fake_A = 0  # L1 loss between real signal A and fake signal A
            test_L1_real_fake_B = 0  # L1 loss between real signal B and fake signal B

            for sig_A, sig_B in test_loader:
                # convert to float16
                sig_A = sig_A.float()
                sig_B = sig_B.float()

                # move to GPU
                sig_A = sig_A.to(DEVICE)
                sig_B = sig_B.to(DEVICE)

                fake_B = gen_B(sig_A)
                fake_A = gen_A(sig_B)

        
                # calculate l1 loss of fake signals and real signals
                test_L1_real_fake_B = l1(sig_B, fake_B)   # l1(sig_B, fake_B)
                test_L1_real_fake_A = l1(sig_A, fake_A)   # l1(sig_A, fake_A)
                
                L1_realA_fakeB = l1(sig_A, fake_B)
                L1_realB_fakeA = l1(sig_B, fake_A)
                
                # ----- discriminator loss ----- #
                test_Discriminator_A_loss = mse(disc_A(sig_A), torch.ones_like(disc_A(sig_A))) + mse(disc_A(fake_A), torch.zeros_like(disc_A(fake_A)))
                test_Discriminator_B_loss = mse(disc_B(sig_B), torch.ones_like(disc_B(sig_B))) + mse(disc_B(fake_B), torch.zeros_like(disc_B(fake_B)))
                
                # ----- total discriminator loss ----- #
                test_Total_Discriminator_loss = test_Discriminator_A_loss + test_Discriminator_B_loss

                test_Total_Generator_loss = 0 # initialize
                # calculate adversarial loss
                if ADVERSARIAL:
                    test_Adversaral_loss_B = mse(disc_B(fake_B), torch.ones_like(disc_B(fake_B))) #was ones_like before
                    test_Adversaral_loss_A = mse(disc_A(fake_A), torch.ones_like(disc_A(fake_A)))

                    test_Total_Generator_loss += test_Adversaral_loss_A * LAMBDA_ADVERSARIAL + test_Adversaral_loss_B * LAMBDA_ADVERSARIAL 

                # ----- cycle loss ----- #
                if CYCLE:
                    
                    test_Cycle_consistency_loss_A = l1(gen_A(gen_B(sig_A)), sig_A)  # forward cycle loss
                    test_Cycle_consistency_loss_B = l1(gen_B(gen_A(sig_B)), sig_B)  # Backward cycle loss


                    test_Total_Generator_loss += LAMBDA_CYCLE * (cycle_B_loss + cycle_A_loss)
                    
                    # cycle_B = gen_B(fake_A)  # fake_A = gen_B2A(sig_B)
                    # cycle_A = gen_A(fake_B)  # fake_B = gen_A2B(sig_A)
                    # test_Cycle_consistency_loss_B = l1(sig_B, cycle_B)
                    # test_Cycle_consistency_loss_A = l1(sig_A, cycle_A)

                    # test_Total_Generator_loss += test_Cycle_consistency_loss_B * LAMBDA_CYCLE + test_Cycle_consistency_loss_A * LAMBDA_CYCLE

                # ----- supervise loss ------ #
                if SUPERVISED:
                    test_sup_A_loss = mse(sig_A, fake_A)
                    test_sup_B_loss = mse(sig_B, fake_B)

                    test_Total_Generator_loss += LAMBDA_SUPERVISED * (test_sup_A_loss + test_sup_B_loss)

                # ----- identity loss ----- #
                if IDENTITY:
                    
                    identity_loss = LAMBDA_IDENTITY * (LAMBDA_CYCLE * (l1(gen_B(sig_B), sig_B) + l1(gen_A(sig_A), sig_A)))
                    test_Total_Generator_loss += identity_loss
                #     id_B = gen_B(sig_B)
                #     id_A = gen_A(sig_A)
                #     id_B_loss = l1(sig_A, id_B)
                #     id_A_loss = l1(sig_B, id_A)

                #     test_Total_Generator_loss += id_B_loss * LAMBDA_IDENTITY + id_A_loss * LAMBDA_IDENTITY
                    
                # # ----- discriminator loss ----- #
                # test_Discriminator_A_loss = mse(disc_A(sig_A), torch.ones_like(disc_A(sig_A))) + mse(disc_A(fake_A), torch.zeros_like(disc_A(fake_A)))
                # test_Discriminator_B_loss = mse(disc_B(sig_B), torch.ones_like(disc_B(sig_B))) + mse(disc_B(fake_B), torch.zeros_like(disc_B(fake_B)))
                
                # # ----- total discriminator loss ----- #
                # test_Total_Discriminator_loss = test_Discriminator_A_loss + test_Discriminator_B_loss

                # save losses
                test_losses['Discriminator A loss'].append(test_Discriminator_A_loss.item())
                test_losses['Discriminator B loss'].append(test_Discriminator_B_loss.item())
                test_losses['Total Discriminator loss'].append(test_Total_Discriminator_loss.item())
                if ADVERSARIAL:
                    test_losses['Adversaral loss A'].append(test_Adversaral_loss_A.item())
                    test_losses['Adversaral loss B'].append(test_Adversaral_loss_B.item())
                if CYCLE:
                    test_losses['Cycle consistency loss A'].append(test_Cycle_consistency_loss_A.item())
                    test_losses['Cycle consistency loss B'].append(test_Cycle_consistency_loss_B.item())
                if SUPERVISED:
                    test_losses['Supervised loss A'].append(test_sup_A_loss.item())
                    test_losses['Supervised loss B'].append(test_sup_B_loss.item())
                if IDENTITY:
                    test_losses['identity_loss'].append(identity_loss.item())
                
                test_losses['Total Generator loss'].append(test_Total_Generator_loss.item())
                test_losses['L1 loss between real signal A and fake signals A'].append(test_L1_real_fake_A.item())
                test_losses['L1 loss between real signal B and fake signals B'].append(test_L1_real_fake_B.item())
                test_losses['L1_realA_fakeB'].append(L1_realA_fakeB.item())
                test_losses['L1_realB_fakeA'].append(L1_realB_fakeA.item())


            #  ------------------------------------- #   
            #  ------- Generate fake signals ------- #
            #  ------------------------------------- #
            
            # Generate fake signals after the last epoch
            
            if (epoch+1) % GENERATION_AFTER_EPOCH == 0:
                print('Generate fake signals')
                # generate fake signals 10 times
                #utils.save_predictions(gen_loader, gen_B, gen_A, fake_A, fake_B, DEVICE, mse)
                idx =0
                for sig_A, sig_B in gen_loader:
                    if idx == 2:
                        break

                    sig_A = sig_A.float()
                    sig_B = sig_B.float()
                    sig_A = sig_A.to(DEVICE)
                    sig_B = sig_B.to(DEVICE)

                    fake_B = gen_B(sig_A)
                    fake_A = gen_A(sig_B)

                    # plot generated signals and real signals
                    #reshape to 1D
                    fake_B = fake_B.reshape(-1)
                    fake_A = fake_A.reshape(-1)
                    sig_A = sig_A.reshape(-1)
                    sig_B = sig_B.reshape(-1)

                    fig, ax = plt.subplots(2, 1, figsize=(10, 8))
                    ax[0].plot(sig_A.cpu().numpy(), label= 'Real signal A')
                    ax[0].plot(fake_A.cpu().numpy(), label= 'Generated signal A')
                    ax[0].set_xlabel('Epoch')
                    ax[0].set_ylabel('Loss')
                    ax[0].legend()

                    ax[1].plot(sig_B.cpu().numpy(), label= 'Real signal B')
                    ax[1].plot(fake_B.cpu().numpy(), label= 'Generated signal B')
                    ax[1].set_xlabel('Epoch')
                    ax[1].set_ylabel('Loss')
                    ax[1].legend()

                    # plot generated signals and real signals
                    plt.figure(figsize=(15, 5))
                    plt.plot(sig_A.cpu().detach().numpy(), label= 'Real signal A')
                    plt.plot(sig_B.cpu().detach().numpy(), label= 'Real signal B')
                    plt.plot(fake_A.cpu().detach().numpy(), label= 'Generated signal A')
                    plt.plot(fake_B.cpu().detach().numpy(), label= 'Generated signal B')
                    plt.title('Generated signals vs real signals')
                    plt.legend()

                    idx += 1

        # activate training mode again
        disc_A.train()  
        disc_B.train()
        gen_B.train()
        gen_A.train()

    # scheduler step if epoch > LR_DECAY_AFTER_EPOCH
    if (epoch+1) >= LR_DECAY_AFTER_EPOCH:
        disc_scheduler.step()
        gen_scheduler.step()

    #  ------------------------------------- #   
    #  ------------- Checkpoint ------------ #
    #  ------------------------------------- #

    if (epoch+1) % SAVE_EPOCH == 0 and SAVE_MODEL == True:

        gen_B_filename = 'Checkpoints/GenB_FeatureNames_{}_Target_[{}]_BatchSize_[{}]_LR_[{}]_Epoch_[{}].pth.tar'.format(feature_names, target, BATCH_SIZE, LEARNING_RATE, epoch+1)
        gen_A_filename = 'Checkpoints/GenA_FeatureNames_{}_Target_[{}]_BatchSize_[{}]_LR_[{}]_Epoch_[{}].pth.tar'.format(feature_names, target, BATCH_SIZE, LEARNING_RATE, epoch+1)
        disc_B_filename = 'Checkpoints/DiscB_FeatureNames_{}_Target_[{}]_BatchSize_[{}]_LR_[{}]_Epoch_[{}].pth.tar'.format(feature_names, target, BATCH_SIZE, LEARNING_RATE, epoch+1)
        disc_A_filename = 'Checkpoints/DiscA_FeatureNames_{}_Target_[{}]_BatchSize_[{}]_LR_[{}]_Epoch_[{}].pth.tar'.format(feature_names, target, BATCH_SIZE, LEARNING_RATE, epoch+1)

        utils.save_checkpoint(gen_B, opt_gen, path=gen_B_filename)
        utils.save_checkpoint(gen_A, opt_gen, path=gen_A_filename)
        utils.save_checkpoint(disc_B, opt_disc, path=disc_B_filename)
        utils.save_checkpoint(disc_A, opt_disc, path=disc_A_filename)

print('Training finished')

In [ ]:
# ----------------------------------- #
# -------------- PLOT --------------- #
# ----------------------------------- #

# Plot training losses in different subplots

fig, ax = plt.subplots(11, 1, figsize=(12, 24))
ax[0].plot(train_losses['Discriminator A loss'], label= 'Discriminator A loss (Training)')
ax[0].plot(train_losses['Discriminator B loss'], label= 'Discriminator B loss (Training)')
ax[0].plot(train_losses['Total Discriminator loss'], label= 'Total Discriminator loss (Training)')
ax[0].set_xlabel('Epoch')
ax[0].set_ylabel('Loss')
ax[0].legend()
#ax[0].set_title('Training Discriminator Loss')

ax[1].plot(train_losses['Adversaral loss A'], label= 'Adversaral loss A (Training)')
ax[1].plot(train_losses['Adversaral loss B'], label= 'Adversaral loss B (Training)')
ax[1].set_xlabel('Epoch')
ax[1].set_ylabel('Loss')
ax[1].legend()
#ax[1].set_title('Training Adversarial Loss')
ax[2].plot(train_losses['Cycle consistency loss A'], label= 'Cycle consistency loss A (Training)')
ax[2].plot(train_losses['Cycle consistency loss B'], label= 'Cycle consistency loss B (Training)')
ax[2].set_xlabel('Epoch')
ax[2].set_ylabel('Loss')
ax[2].legend()
#ax[2].set_title('Training Cycle Consistency Loss')
ax[3].plot(train_losses['Total Generator loss'], label= 'Total Generator loss (Training)')
ax[3].set_xlabel('Epoch')
ax[3].set_ylabel('Loss')
ax[3].legend()
#ax[3].set_title('Training Total Generator Loss')
ax[4].plot(train_losses['Supervised loss A'], label = 'Supervised loss A (Training)')
ax[4].plot(train_losses['Supervised loss B'], label = 'Supervised loss B (Training)')
ax[4].set_xlabel('Epoch')
ax[4].set_ylabel('Loss')
ax[4].legend()

# Plot test losses in different subplots

ax[5].plot(test_losses['Discriminator A loss'], label= 'Discriminator A loss (Test)')
ax[5].plot(test_losses['Discriminator B loss'], label= 'Discriminator B loss (Test)')
ax[5].plot(test_losses['Total Discriminator loss'], label= 'Total Discriminator loss (Test)')
ax[5].set_xlabel('Epoch')
ax[5].set_ylabel('Loss')
ax[5].legend()
#ax[4].set_title('Test Discriminator Loss')

ax[6].plot(test_losses['Adversaral loss A'], label= 'Adversaral loss A (Test)')
ax[6].plot(test_losses['Adversaral loss B'], label= 'Adversaral loss B (Test)')
ax[6].set_xlabel('Epoch')
ax[6].set_ylabel('Loss')
ax[6].legend()
#ax[5].set_title('Test Adversarial Loss')

ax[7].plot(test_losses['Cycle consistency loss A'], label= 'Cycle consistency loss A (Test)')
ax[7].plot(test_losses['Cycle consistency loss B'], label= 'Cycle consistency loss B (Test)')
ax[7].set_xlabel('Epoch')
ax[7].set_ylabel('Loss')
ax[7].legend()
#ax[6].set_title('Test Cycle Consistency Loss')

ax[8].plot(test_losses['Supervised loss A'], label = 'Supervised loss A (Training)')
ax[8].plot(test_losses['Supervised loss B'], label = 'Supervised loss B (Training)')
ax[8].set_xlabel('Epoch')
ax[8].set_ylabel('Loss')
ax[8].legend()

ax[9].plot(test_losses['Total Generator loss'], label= 'Total Generator loss (Test)')
ax[9].set_xlabel('Epoch')
ax[9].set_ylabel('Loss')
ax[9].legend()
#ax[7].set_title('Test Total Generator Loss')

ax[10].plot(test_losses['L1 loss between real signal A and fake signals A'], label= 'L1 loss between real signal A and fake signals A (Test)')
ax[10].plot(test_losses['L1 loss between real signal B and fake signals B'], label= 'L1 loss between real signal B and fake signals B (Test)')
ax[10].set_xlabel('Epoch')
ax[10].set_ylabel('Loss')
ax[10].legend()
#ax[8].set_title('Test L1 Loss')

plt.show()

print('Selected losses for the test dataset after the last epoch:\n')
print('\nL1 loss between real signal A and fake signals A: ', test_losses['L1 loss between real signal A and fake signals A'][-1])
print('\nL1 loss between real signal B and fake signals B: ', test_losses['L1 loss between real signal B and fake signals B'][-1])
print('\nL1 loss between real signal A and fake signal B: ', test_losses['L1_realA_fakeB'][-1])
print('\nL1 loss between real signal B and fake signal A: ', test_losses['L1_realB_fakeA'][-1])

print('\nDiscriminator A loss: ', test_losses['Discriminator A loss'][-1])
print('\nDiscriminator B loss: ', test_losses['Discriminator B loss'][-1])
print('\nTotal Discriminator loss: ', test_losses['Total Discriminator loss'][-1])
if ADVERSARIAL:
    print('\nAdversaral loss A: ', test_losses['Adversaral loss A'][-1])
    print('\nAdversaral loss B: ', test_losses['Adversaral loss B'][-1])
if CYCLE:
    print('\nCycle consistency loss A: ', test_losses['Cycle consistency loss A'][-1])
    print('\nCycle consistency loss B: ', test_losses['Cycle consistency loss B'][-1])
if SUPERVISED:
    print('\nSupervised loss A: ', test_losses['Supervised loss A'][-1])
    print('\nSupervised loss B: ', test_losses['Supervised loss B'][-1])
print('\nTotal Generator loss: ', test_losses['Total Generator loss'][-1])

Selected losses for the test dataset after the last epoch:


L1 loss between real signal A and fake signals A:  0.5614745020866394

L1 loss between real signal B and fake signals B:  0.20988085865974426

Discrminator A loss:  0.43891608715057373

Discrminator B loss:  0.49221402406692505

Total Discriminator loss:  0.9311301112174988

Adversaral loss A:  0.2902804911136627

Adversaral loss B:  0.2568780779838562

Cycle consistency loss A:  0.4262011647224426

Cycle consistency loss B:  0.06283529847860336

Supervised loss A:  0.5173115730285645

Supervised loss B:  0.07597765326499939

Total Generator loss:  11.370415687561035

In [ ]:
# # load csv
# df = pd.read_csv('generated_signals.csv')
# df.drop('Unnamed: 0', axis=1, inplace=True)

# # plot df
# df.plot(figsize=(12, 6), title='Generated Signals')
# # legend
# plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))


# # save plot
# plt.savefig('{}_to_{}.png'.format(SIG_A, SIG_B), bbox_inches='tight')

In [ ]:
# # load csv
# df = pd.read_csv('generated_signals.csv')
# df.drop('Unnamed: 0', axis=1, inplace=True)

# plt.figure(figsize=(20, 5))
# plt.plot(df['sig_A'], label= 'sig_A')
# plt.plot(df['fake_B'], label= 'fake_B')
# plt.legend()
# plt.show()